In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
from torch import nn
import random as rnd
import pickle

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
with open('gdrive/MyDrive/arabic_letters.pickle', 'rb') as file:
    vocab_list = list(pickle.load(file))

padd_char = '#'

vocab_list.append(padd_char)
vocab = { char: i for i, char in enumerate(vocab_list) }

with open('gdrive/MyDrive/diacritic2id.pickle', 'rb') as file:
    labels_map = pickle.load(file)

labels_map[padd_char] = len(labels_map)

print(vocab)
print(labels_map)


{'ل': 0, 'م': 1, 'ي': 2, 'ع': 3, 'آ': 4, 'ه': 5, 'ك': 6, 'إ': 7, 'ت': 8, 'ؤ': 9, 'ر': 10, 'ذ': 11, 'ج': 12, 'س': 13, 'ث': 14, 'ة': 15, 'ى': 16, 'ظ': 17, 'خ': 18, 'ش': 19, 'ء': 20, 'ص': 21, 'ض': 22, 'ئ': 23, 'غ': 24, 'ز': 25, 'و': 26, 'ح': 27, 'ن': 28, 'ب': 29, 'ا': 30, 'ف': 31, 'أ': 32, 'ق': 33, 'ط': 34, 'د': 35, '#': 36}
{'َ': 0, 'ً': 1, 'ُ': 2, 'ٌ': 3, 'ِ': 4, 'ٍ': 5, 'ْ': 6, 'ّ': 7, 'َّ': 8, 'ًّ': 9, 'ُّ': 10, 'ٌّ': 11, 'ِّ': 12, 'ٍّ': 13, '': 14, '#': 15}


In [4]:
class CustomDataset(torch.utils.data.Dataset):

  def __init__(self, x, y, pad):
    """
    This is the constructor of the NERDataset
    Inputs:
    - x: a list of lists where each list contains the ids of the tokens
    - y: a list of lists where each list contains the label of each token in the sentence
    - pad: the id of the <PAD> token (to be used for padding all sentences and labels to have the same length)
    """
    ##################### TODO: create two tensors one for x and the other for labels ###############################
    # pass
    self.x_tensor = nn.utils.rnn.pad_sequence(
      [torch.tensor([ord(char) for char in sentence]) for sentence in x],
      batch_first = True,
      padding_value = ord(pad)
    )
    self.y_tensor = nn.utils.rnn.pad_sequence(
      [torch.tensor(labels) for labels in y],
      batch_first = True,
      padding_value = labels_map[pad]
    )
    #################################################################################################################

  def __len__(self):
    """
    This function should return the length of the dataset (the number of sentences)
    """
    ###################### TODO: return the length of the dataset #############################
    # pass
    return len(self.x_tensor)
    ###########################################################################################

  def __getitem__(self, idx):
    """
    This function returns a subset of the whole dataset
    """
    ###################### TODO: return a tuple of x and y ###################################
    # pass
    return (self.x_tensor[idx], self.y_tensor[idx])
    ##########################################################################################

In [5]:
def prepare_dataloader():
  with open('gdrive/MyDrive/sentences_list.pickle', 'rb') as file:
    sentences = pickle.load(file)

  with open('gdrive/MyDrive/labels_list.pickle', 'rb') as file:
    labels = pickle.load(file)

  print(sentences[0])
  print(labels[0])
  return CustomDataset(sentences, labels, padd_char)


In [6]:
batch_size = 64
# mini_sentences = t_sentences[0: 8]
# mini_labels = t_labels[0: 8]
# mini_dataset = NERDataset(mini_sentences, mini_labels, vocab['<PAD>'])
# dummy_dataloader = torch.utils.data.DataLoader(mini_dataset, batch_size=5)
# dg = iter(dummy_dataloader)
# X1, Y1 = next(dg)
# X2, Y2 = next(dg)
# print(Y1.shape, X1.shape, Y2.shape, X2.shape)
# print(X1[0][:], "\n", Y1[0][:])

In [7]:
import torch.nn as nn

class DiacriticLSTM(nn.Module):
    def __init__(self, vocab_size=35181, embedding_dim=250, hidden_size=100, num_layers=2, n_classes=len(labels_map), dropout_rate=0.5):
        super(DiacriticLSTM, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=vocab[padd_char])
        self.lstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True
        )
        self.dropout = nn.Dropout(dropout_rate)
        # self.batch_norm = nn.BatchNorm1d(hidden_size * 2)
        self.linear = nn.Linear(hidden_size * 2, n_classes)

    def forward(self, sentences):
        embedded = self.embedding(sentences)
        lstm_out, _ = self.lstm(embedded)
        # lstm_out = self.batch_norm(lstm_out)
        lstm_out = self.dropout(lstm_out)
        final_output = self.linear(lstm_out)
        return final_output


# Training

In [8]:
from torch.optim import lr_scheduler
import time

def train(model, train_dataset, batch_size=128, epochs=5, learning_rate=0.01):
    """
    This function implements the training logic
    Inputs:
    - model: the model ot be trained
    - train_dataset: the training set of type NERDataset
    - batch_size: integer represents the number of examples per step
    - epochs: integer represents the total number of epochs (full training pass)
    - learning_rate: the learning rate to be used by the optimizer
    """

    # (1) create the dataloader of the training set (make the shuffle=True)
    train_dataloader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True
    )

    # (2) make the criterion cross entropy loss
    criterion = torch.nn.CrossEntropyLoss()

    # (3) create the optimizer (Adam)

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)


    # (4) create a learning rate scheduler
    scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

    # GPU configuration
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    if use_cuda:
        model = model.cuda()
        criterion = criterion.cuda()

    for epoch_num in range(epochs):
        total_acc_train = 0
        total_loss_train = 0

        model.train()  # Set model to training mode

        for train_input, train_label in tqdm(train_dataloader):

            # (5) move the train input to the device
            train_input = train_input.to(device)

            # (6) move the train label to the device
            train_label = train_label.to(device)

            # (7) do the forward pass
            output = model(train_input)

            # (8) loss calculation (you need to think in this part how to calculate the loss correctly)
            batch_loss = criterion(output.view(-1, output.shape[-1]), train_label.view(-1))

            # (9) append the batch loss to the total_loss_train
            total_loss_train += batch_loss.item()

            # (10) calculate the batch accuracy (just add the number of correct predictions)
            acc = (output.argmax(dim=-1) == train_label).sum().item()
            total_acc_train += acc

            # (11) zero your gradients
            optimizer.zero_grad()

            # (12) do the backward pass
            batch_loss.backward()

            # (13) update the weights with your optimizer
            optimizer.step()

            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)


        # (14) step the learning rate scheduler
        scheduler.step()

        # epoch loss
        epoch_loss = total_loss_train / len(train_dataset)

        # (15) calculate the accuracy
        epoch_acc = total_acc_train / (len(train_dataset) * train_dataset[0][0].shape[0])

        print(
            f'Epochs: {epoch_num + 1} | Train Loss: {epoch_loss} \
            | Train Accuracy: {epoch_acc}\n')

    ##############################################################################################################




In [9]:
# train_dataset = NERDataset(t_sentences, t_labels, vocab['<PAD>'])
# val_dataset = NERDataset(v_sentences, v_labels, vocab['<PAD>'])
# test_dataset = NERDataset(test_sentences, test_labels, vocab['<PAD>'])
train_dataset = prepare_dataloader()

قوله أو قطع الأول يده إلخ قال الزركشي
[0, 6, 2, 2, 14, 0, 6, 14, 0, 0, 0, 14, 14, 6, 0, 8, 2, 14, 0, 0, 2, 14, 14, 0, 6, 14, 0, 14, 0, 14, 14, 14, 8, 6, 0, 4, 10]


In [10]:
model = DiacriticLSTM()

In [11]:
import torch
torch.cuda.empty_cache()

In [12]:
train(model, train_dataset)

100%|██████████| 402/402 [02:41<00:00,  2.48it/s]


Epochs: 1 | Train Loss: 0.0005076735432081662             | Train Accuracy: 0.9786629038652119



100%|██████████| 402/402 [02:49<00:00,  2.37it/s]


Epochs: 2 | Train Loss: 0.00022392445156034434             | Train Accuracy: 0.9907587807969958



100%|██████████| 402/402 [02:49<00:00,  2.37it/s]


Epochs: 3 | Train Loss: 0.00019395484157345728             | Train Accuracy: 0.9920345932780432



100%|██████████| 402/402 [02:49<00:00,  2.37it/s]


Epochs: 4 | Train Loss: 0.00017911656285956592             | Train Accuracy: 0.9926489587518443



100%|██████████| 402/402 [02:49<00:00,  2.37it/s]

Epochs: 5 | Train Loss: 0.00016922134767650423             | Train Accuracy: 0.9930587743778897



#### Expected train accuracy after 5 epochs to be above 0.99

# Evaluation

In [13]:
def evaluate(model, test_dataset, batch_size=256):
  """
  This function takes a NER model and evaluates its performance (accuracy) on a test data
  Inputs:
  - model: a NER model
  - test_dataset: dataset of type NERDataset
  """
  ########################### TODO: Replace the Nones in the following code ##########################

  # (1) create the test data loader
  test_dataloader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size = batch_size,
    shuffle = False
  )

  # GPU Configuration
  use_cuda = torch.cuda.is_available()
  device = torch.device("cuda" if use_cuda else "cpu")
  if use_cuda:
    model = model.cuda()

  total_acc_test = 0

  # (2) disable gradients
  with torch.no_grad():

    for test_input, test_label in tqdm(test_dataloader):
      # (3) move the test input to the device
      test_label = test_label.to(device)

      # (4) move the test label to the device
      test_input = test_input.to(device)

      # (5) do the forward pass
      output = model(test_input)

      # accuracy calculation (just add the correct predicted items to total_acc_test)
      acc = (torch.argmax(output, dim=-1) == test_label).sum().item()
      total_acc_test += acc

    # (6) calculate the over all accuracy
    total_acc_test /= (len(test_dataset) * test_dataset[0][0].shape[0])
  ##################################################################################################


  print(f'\nTest Accuracy: {total_acc_test}')

In [14]:
with open('gdrive/MyDrive/val_sentences_list.pickle', 'rb') as file:
  test_sentences = pickle.load(file)

with open('gdrive/MyDrive/val_labels_list.pickle', 'rb') as file:
  test_labels = pickle.load(file)

print(test_sentences[0])
print(test_labels[0])
test_dataset = CustomDataset(test_sentences, test_labels, padd_char)

قوله ولا تكره ضيافته
[0, 6, 2, 2, 14, 0, 0, 14, 14, 2, 6, 0, 2, 14, 4, 0, 14, 0, 2, 2]


In [15]:
evaluate(model, test_dataset)

100%|██████████| 11/11 [00:02<00:00,  4.91it/s]


Test Accuracy: 0.9907391194194031


## Test

In [16]:
def test(model, test_dataset, batch_size=256):
  """
  This function takes a NER model and evaluates its performance (accuracy) on a test data
  Inputs:
  - model: a NER model
  - test_dataset: dataset of type NERDataset
  """
  ########################### TODO: Replace the Nones in the following code ##########################

  # (1) create the test data loader
  test_dataloader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size = batch_size,
    shuffle = False
  )

  # GPU Configuration
  use_cuda = torch.cuda.is_available()
  device = torch.device("cuda" if use_cuda else "cpu")
  if use_cuda:
    model = model.cuda()

  pred = []

  # (2) disable gradients
  with torch.no_grad():

    for test_input, test_label in tqdm(test_dataloader):
      # (3) move the test input to the device
      test_label = test_label.to(device)

      # (4) move the test label to the device
      test_input = test_input.to(device)
      # print('INPUT:', test_input)
      # (5) do the forward pass
      output = model(test_input)
      # print('OUTPUT:', output)
      p = torch.argmax(output, dim=-1)
      # print('PREDECTION:', p)
      pred.append(p)

  return pred

In [38]:
with open('gdrive/MyDrive/test_sentences_list_1.pickle', 'rb') as file:
  test_sentences_leaderboard = pickle.load(file)

with open('gdrive/MyDrive/test_labels_list_1.pickle', 'rb') as file:
  test_labels_leaderboard = pickle.load(file)

print(test_sentences_leaderboard[0])
print(test_labels_leaderboard[0])
test_dataset_leaderboard = CustomDataset(test_sentences_leaderboard, test_labels_leaderboard, padd_char)

صوتقديمالحلقأوالإفاضةعلىالرميشهذابالجرمعطوفعلىمايوجبالدموهوقولهفيمامركتأخيرالحلقلبلدهوالمعنىأنهإذاقدمالحلقعلىرميجمرةالعقبةفإنهتلزمهالفديةلوقوعهقبلشيءمنالتحللكمافيالمدونةلاهديكمايعطيهكلامالمؤلفلأنالدمإنماينصرفللهديفإذارمىالعقبةأمرالموسىعلىرأسهلأنالحلقالأولوقعقبلمحلهوكذلكيلزمهالهديإذاقدمطوافالإفاضةعلىرميجمرةالعقبةمعالإجزاءعلىالمشهوروكلامالمؤلفيصدقبتقديمالإفاضةعلىيومالنحروليسبمرادلأنفعلالإفاضةقبليومالنحركلافعللأنهفعللهاقبلوقتهاولوقدمكلامنالإفاضةوالحلقعلىالرميلوجبفيهمافديةوهديثمإنالترتيببينكلمنهماوبينالرميواجبإذلوكانمستحبالماوجبفيهشيءوهوظاهرلأنالرميهوالتحللالأصغرصلاإنخالففيغيرشأيلاإنخالفعمداأونسياناأوجهلافيغيرماتقدمبأنحلققبلأنيذبحأونحرقبلأنيرميأوقدمالإفاضةعلىالنحرأوعلىالحلقأوعليهمافإنهلادم
[14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 

In [39]:
pred = test(model, test_dataset_leaderboard)

100%|██████████| 1/1 [00:00<00:00, 116.61it/s]


In [40]:
print(len(pred[0][0]))

925


In [41]:
numpy_pred = [batch.cpu().detach().numpy() for batch in pred]
final_pred = np.concatenate(numpy_pred)

In [42]:
print(final_pred.shape)
total = 0
for sentence in final_pred:
  for char in sentence:
    if char != 15:
      total += 1
print(total)
print(len(test_sentences_leaderboard[0]))

(200, 925)
34074
694


In [43]:
import csv

index = 0

with open('test_prediction.csv', 'a', newline='') as csvfile:
  fieldnames = ['ID', 'label']
  writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
  if csvfile.tell() == 0:
      writer.writeheader()

  for i, sentence in enumerate(test_sentences_leaderboard):
    for j, char in enumerate(sentence):
      if char != ' ':
        writer.writerow({'ID': index, 'label': final_pred[i][j]})
        index += 1

In [44]:
print(index)

34074
